In [18]:
import tensorflow as tf
import numpy as np

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [19]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [20]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [7]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

In [21]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(inputs=X, units=n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(inputs=hidden1, units=n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(inputs=hidden2, units=n_outputs, activation=None, name="outputs")

In [28]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[12]|outputs")

In [29]:
train_vars

[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'hidden2/kernel:0' shape=(300, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'outputs/kernel:0' shape=(100, 10) dtype=float32_ref>,
 <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]

In [31]:
for var in tf.global_variables():
    print(var.name)

hidden1/kernel:0
hidden1/bias:0
hidden2/kernel:0
hidden2/bias:0
outputs/kernel:0
outputs/bias:0


In [22]:
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))

In [23]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [24]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) # K表示每个样本的预测结果的前K个最大的数里面是否含有target中的值
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [25]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [26]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [32]:
n_epochs = 80
batch_size = 50

In [33]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        if epoch % 20 == 0:
            print(epoch, "Train accuracy: ", acc_train, "Test accuracy: ", acc_test)
        
    save_path = saver.save(sess, "C:\Boyuan\MyPython\my_model_final.ckpt")

0 Train accuracy:  0.9 Test accuracy:  0.905


20 Train accuracy:  1.0 Test accuracy:  0.9718


40 Train accuracy:  1.0 Test accuracy:  0.9789


60 Train accuracy:  1.0 Test accuracy:  0.9794


In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "C:\Boyuan\MyPython\my_model_final.ckpt")
    X_new_scaled = []
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)